In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

from VAEmodel import *

# Load the Data

In [2]:
# Load the processed MSA data
import pickle
from pathlib import Path

DATA_DIR = Path("../../data")
# REF_SEQ_ID = "PF00041"
REF_SEQ_ID = "PF00288"
AA_TYPES = [' ',
      'R', 'H', 'K',
      'D', 'E',
      'S', 'T', 'N', 'Q',
      'C', 'G', 'P',
      'A', 'V', 'I', 'L', 'M', 'F', 'Y', 'W']

enumd_msa = np.load(DATA_DIR / "processed" / "enumd_mtx_{}.npy".format(REF_SEQ_ID))
print("enumd MSA shape: ", enumd_msa.shape)
seq_weights = np.load(DATA_DIR / "processed" / "seq_weights_{}.npy".format(REF_SEQ_ID))
print("seq weights shape: ", seq_weights.shape)
seqs_infos = pd.read_csv(DATA_DIR / "processed" / "seq_infos_{}.csv".format(REF_SEQ_ID))
print("seq infos shape: ", seqs_infos.shape)

msa = MSA_Dataset(enumd_msa, seq_weights, seqs_infos, MSA_to_OneHot)

# check length = num of seqs
print("Number of sequences in loaded alignment: ", len(msa))
print("Length of loaded alignment (final number of columns/a.a. positions in alignment for each sequence): ", enumd_msa.shape[1])

enumd MSA shape:  (20151, 59)
seq weights shape:  (20151,)
seq infos shape:  (20151, 4)
Number of sequences in loaded alignment:  20151
Length of loaded alignment (final number of columns/a.a. positions in alignment for each sequence):  59


# Initialize Model

In [3]:
# import torch_xla.core.xla_model as xm
# dev = xm.xla_device()

print("GPU available: ", torch.cuda.is_available())
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 2-dimensional latent space
print(f"flattened sequences {enumd_msa.shape[1] * len(AA_TYPES)} dimensions")
vae = VAE(len(AA_TYPES), 2, enumd_msa.shape[1] * len(AA_TYPES), [200, 200])
vae.to(dev)

GPU available:  True
flattened sequences 1239 dimensions


VAE(
  (encoder_comm_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=1239, out_features=200, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=200, out_features=200, bias=True)
      (1): Tanh()
    )
  )
  (enc_mu): Linear(in_features=200, out_features=2, bias=True)
  (enc_logvars): Linear(in_features=200, out_features=2, bias=True)
)

In [4]:
first_latent = vae.encode(msa[0][0].to(dev))
print("initial latent projection of first sequence: ", first_latent)

applying transform  <VAEmodel.MSA_to_OneHot object at 0x7f8e2231b1f0>
before transform: tensor([14, 16, 19, 11,  8, 15, 12,  8, 11,  6, 11, 16,  6,  6,  6, 13,  6, 16,
         5, 14, 16, 17, 11,  6, 15, 16,  3,  5, 17, 19, 11, 16,  5, 13, 16, 15,
        11,  9, 19,  6,  5,  8,  8, 19,  8, 17,  8, 10, 11, 15, 17,  4,  9, 18,
        13,  6, 13, 17, 11])
initial latent projection of first sequence:  (tensor([ 0.0244, -0.0719], device='cuda:0', grad_fn=<AddBackward0>), tensor([1.0570, 1.0610], device='cuda:0', grad_fn=<ExpBackward0>))


# Set up train, validation, test

In [5]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

train_size = int(0.8 * len(msa))
train_set, val_set = torch.utils.data.random_split(msa, [train_size, len(msa) - train_size])
print("num of seqs in train set: ", len(train_set))
print("num of seqs in validation set: ", len(val_set))

# 320k / 64 = 5000 batches per epoch
batch_size = 64
weight_decay = 1e-3
# good rule of thumb: start with epochs := 3 * num of columns in data
# also thanks to [How to choose a batch size and the number of epochs while training a NN](https://stats.stackexchange.com/a/529405)
epochs = 3 * enumd_msa.shape[1] 

print("Training")
print('-' * 20)
print("Using Adam Optimizer; Hyperparameters")
# print("batch size: ", batch_size, " sequences")
print("sampling one at a time, batch size = 1")
# print("learning rate: ", learn_rate)
print("number of epochs: ", epochs)

num of seqs in train set:  16120
num of seqs in validation set:  4031
Training
--------------------
Using Adam Optimizer; Hyperparameters
sampling one at a time, batch size = 1
number of epochs:  177


In [6]:
train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)

optimizer = optim.Adam(vae.parameters(),
                       weight_decay=weight_decay)

tb_logger = SummaryWriter()

In [7]:
running_loss = []
for epoch in range(epochs):
    # just for convenience of using DataLoader,
    # only one sample => i loop through i=0
    for i, data in enumerate(train_loader):
        seq, weight, seq_info = data
        seq = seq.to(dev)
        weight = weight.to(dev)
        optimizer.zero_grad()
        loss = (-1)*vae.calc_weighted_elbo(seq, weight)
        loss.backward()
        optimizer.step()

    # Gather data and report
    running_loss.append(loss.item())
    if (epoch+1) % 50 == 0:
        print('epoch {} loss: {}'.format(epoch+1, running_loss[epoch]))
        tb_x = epoch*len(train_loader) + i+1
        tb_logger.add_scalar('Loss/train', running_loss[epoch], tb_x)

vae.cpu()
torch.save(vae.state_dict(), DATA_DIR / "models" / "vae_{}_{}_epochs.pt".format(REF_SEQ_ID, epochs))

applying transform  <VAEmodel.MSA_to_OneHot object at 0x7f8e2231b1f0>
before transform: tensor([14, 15,  5, 11,  7, 15, 12,  8, 11, 13, 11, 16,  6,  6,  6, 13,  6, 16,
         5, 16, 16, 14, 11, 14, 17, 16,  5,  4, 16, 19,  4, 16, 16, 14,  3, 15,
        11,  9,  3, 14,  5,  8,  5, 18, 15, 14,  8,  6, 11, 15, 17,  4,  9, 18,
        13, 15, 11, 18, 11])


AttributeError: 'VAE' object has no attribute 'decoder'

### Visualize data flow within model

In [ ]:
# Again, grab a single mini-batch of images
dataiter = iter(train_loader)
seq, weight, seq_infos = next(dataiter)

# add_graph() will trace the sample input through your model,
# and render it as a graph.
tb_logger.add_graph(vae, seq)
tb_logger.flush()